# Shot Probability Model

Many factors affect the chances that a field goal attempt will be successful. While not exhaustive, the following list contains a few:

- Shooting ability of the shooter
- Proximity (and number) of the closest defender(s)
- Shot location
- Shot type

The following introduces the `Player` class of the [py_ball](https://github.com/basketballrelativity/py_ball) package by exploring the `shortchartdetail` and `playbyplay` endpoints of the [stats.nba.com](https://stats.nba.com).

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import time

from py_ball import player, playbyplay

HEADERS = {'Connection': 'close',
           'Host': 'stats.nba.com',
           'Origin': 'http://stats.nba.com',
           'Upgrade-Insecure-Requests': '1',
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2)' + \
                         'AppleWebKit/537.36 (KHTML, like Gecko) ' + \
                         'Chrome/66.0.3359.117 Safari/537.36'}

The `shotchartdetail` endpoint accepts a variety of parameters, but only a few are relevant for shot probability purposes:
- league_id: '00' for the NBA, '10' for the WNBA
- player_id: The unique identifier for the player (0 returns shot data for the whole game)
- game_id: The unique identifier for the game of interest
- season: YYYY-ZZ season year format (This only affects the league averages returned)

In [11]:
league_id = '00' #NBA
player_id = '0' #All players
game_id = '0021700608'
season = '2017-18'

shots = player.Player(headers=HEADERS,
                      endpoint='shotchartdetail',
                      league_id=league_id,
                      player_id=player_id,
                      game_id=game_id,
                      season=season)
shot_df = pd.DataFrame(shots.data['Shot_Chart_Detail'])
shot_df.head(15)

,ACTION_TYPE,EVENT_TYPE,GAME_DATE,GAME_EVENT_ID,GAME_ID,GRID_TYPE,HTM,LOC_X,LOC_Y,MINUTES_REMAINING,...,SHOT_ATTEMPTED_FLAG,SHOT_DISTANCE,SHOT_MADE_FLAG,SHOT_TYPE,SHOT_ZONE_AREA,SHOT_ZONE_BASIC,SHOT_ZONE_RANGE,TEAM_ID,TEAM_NAME,VTM
0,Jump Shot,Made Shot,20180110,7,0021700608,Shot Chart Detail,HOU,226,151,11,...,1,27,1,3PT Field Goal,Right Side Center(RC),Above the Break 3,24+ ft.,1610612745,Houston Rockets,POR
1,Pullup Jump shot,Missed Shot,20180110,9,0021700608,Shot Chart Detail,HOU,123,105,11,...,1,16,0,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612757,Portland Trail Blazers,POR
2,Jump Shot,Missed Shot,20180110,11,0021700608,Shot Chart Detail,HOU,-162,187,11,...,1,24,0,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612745,Houston Rockets,POR
3,Jump Shot,Missed Shot,20180110,13,0021700608,Shot Chart Detail,HOU,144,253,10,...,1,29,0,3PT Field Goal,Right Side Center(RC),Above the Break 3,24+ ft.,1610612757,Portland Trail Blazers,POR
4,Driving Layup Shot,Missed Shot,20180110,15,0021700608,Shot Chart Detail,HOU,-29,6,10,...,1,2,0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612745,Houston Rockets,POR
5,Pullup Jump shot,Missed Shot,20180110,18,0021700608,Shot Chart Detail,HOU,-214,126,10,...,1,24,0,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612745,Houston Rockets,POR
6,Jump Shot,Missed Shot,20180110,20,0021700608,Shot Chart Detail,HOU,-242,62,10,...,1,24,0,3PT Field Goal,Left Side(L),Left Corner 3,24+ ft.,1610612757,Portland Trail Blazers,POR
7,Turnaround Fadeaway shot,Made Shot,20180110,22,0021700608,Shot Chart Detail,HOU,57,88,10,...,1,10,1,2PT Field Goal,Right Side(R),In The Paint (Non-RA),8-16 ft.,1610612745,Houston Rockets,POR
8,Layup Shot,Missed Shot,20180110,23,0021700608,Shot Chart Detail,HOU,-2,62,9,...,1,6,0,2PT Field Goal,Center(C),In The Paint (Non-RA),Less Than 8 ft.,1610612757,Portland Trail Blazers,POR
9,Driving Finger Roll Layup Shot,Missed Shot,20180110,25,0021700608,Shot Chart Detail,HOU,-11,31,9,...,1,3,0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612745,Houston Rockets,POR


In [12]:
list(shot_df)

['ACTION_TYPE',
 'EVENT_TYPE',
 'GAME_DATE',
 'GAME_EVENT_ID',
 'GAME_ID',
 'GRID_TYPE',
 'HTM',
 'LOC_X',
 'LOC_Y',
 'MINUTES_REMAINING',
 'PERIOD',
 'PLAYER_ID',
 'PLAYER_NAME',
 'SECONDS_REMAINING',
 'SHOT_ATTEMPTED_FLAG',
 'SHOT_DISTANCE',
 'SHOT_MADE_FLAG',
 'SHOT_TYPE',
 'SHOT_ZONE_AREA',
 'SHOT_ZONE_BASIC',
 'SHOT_ZONE_RANGE',
 'TEAM_ID',
 'TEAM_NAME',
 'VTM']

Looks like the `shotchartdetail` endpoint has shot location (including zone and coordinates) and type. However, defender and shot clock information do not seem to be available. To calculate shooter ability, the `playerdashboardbyshootingsplits` endpoint may be of use.

In [13]:
shooters = player.Player(headers=HEADERS,
                         endpoint='playerdashboardbyshootingsplits',
                         league_id=league_id,
                         player_id='2772',
                         season=season)
shot_type_df = pd.DataFrame(shooters.data['ShotTypePlayerDashboard'])
shot_type_df.head(10)

,BLKA,BLKA_RANK,CFID,CFPARAMS,EFG_PCT,EFG_PCT_RANK,FG3A,FG3A_RANK,FG3M,FG3M_RANK,...,PCT_AST_3PM,PCT_AST_3PM_RANK,PCT_AST_FGM,PCT_AST_FGM_RANK,PCT_UAST_2PM,PCT_UAST_2PM_RANK,PCT_UAST_3PM,PCT_UAST_3PM_RANK,PCT_UAST_FGM,PCT_UAST_FGM_RANK
0,0,1,50,Alley Oop Dunk Shot,0.000,24,0,8,0,8,...,0.0,8,0.000,21,0.000,15,0.0,5,0.000,18
1,0,1,50,Alley Oop Layup shot,0.000,24,0,8,0,8,...,0.0,8,0.000,21,0.000,15,0.0,5,0.000,18
2,0,1,50,Cutting Dunk Shot,1.000,3,0,8,0,8,...,0.0,8,1.000,1,0.000,15,0.0,5,0.000,18
3,0,1,50,Cutting Finger Roll Layup Shot,0.000,24,0,8,0,8,...,0.0,8,0.000,21,0.000,15,0.0,5,0.000,18
4,2,40,50,Cutting Layup Shot,0.667,16,0,8,0,8,...,0.0,8,1.000,1,0.000,15,0.0,5,0.000,18
5,0,1,50,Driving Bank Hook Shot,0.000,24,0,8,0,8,...,0.0,8,0.000,21,0.000,15,0.0,5,0.000,18
6,0,1,50,Driving Dunk Shot,1.000,3,0,8,0,8,...,0.0,8,0.833,8,0.167,14,0.0,5,0.167,16
7,0,1,50,Driving Finger Roll Layup Shot,0.909,10,0,8,0,8,...,0.0,8,0.800,9,0.200,13,0.0,5,0.200,15
8,0,1,50,Driving Floating Bank Jump Shot,0.750,13,0,8,0,8,...,0.0,8,0.667,11,0.333,11,0.0,5,0.333,11
9,3,43,50,Driving Floating Jump Shot,0.250,23,0,8,0,8,...,0.0,8,0.500,15,0.500,9,0.0,5,0.500,7


In [14]:
list(shot_type_df)

['BLKA',
 'BLKA_RANK',
 'CFID',
 'CFPARAMS',
 'EFG_PCT',
 'EFG_PCT_RANK',
 'FG3A',
 'FG3A_RANK',
 'FG3M',
 'FG3M_RANK',
 'FG3_PCT',
 'FG3_PCT_RANK',
 'FGA',
 'FGA_RANK',
 'FGM',
 'FGM_RANK',
 'FG_PCT',
 'FG_PCT_RANK',
 'GROUP_SET',
 'GROUP_VALUE',
 'PCT_AST_2PM',
 'PCT_AST_2PM_RANK',
 'PCT_AST_3PM',
 'PCT_AST_3PM_RANK',
 'PCT_AST_FGM',
 'PCT_AST_FGM_RANK',
 'PCT_UAST_2PM',
 'PCT_UAST_2PM_RANK',
 'PCT_UAST_3PM',
 'PCT_UAST_3PM_RANK',
 'PCT_UAST_FGM',
 'PCT_UAST_FGM_RANK']

The `shot_type_df` has a `CFPARAMS` field which seems to correspond to `ACTION_TYPE` in `shot_df`.

In [15]:
list(set(shot_df['ACTION_TYPE']))

['Turnaround Fadeaway shot',
 'Driving Finger Roll Layup Shot',
 'Putback Layup Shot',
 'Driving Layup Shot',
 'Dunk Shot',
 'Jump Shot',
 'Alley Oop Dunk Shot',
 'Driving Floating Jump Shot',
 'Running Reverse Layup Shot',
 'Layup Shot',
 'Fadeaway Jump Shot',
 'Pullup Jump shot',
 'Tip Layup Shot',
 'Step Back Jump shot',
 'Reverse Layup Shot',
 'Driving Dunk Shot',
 'Putback Dunk Shot',
 'Cutting Dunk Shot',
 'Floating Jump shot',
 'Driving Reverse Layup Shot',
 'Turnaround Jump Shot',
 'Cutting Layup Shot',
 'Running Layup Shot',
 'Turnaround Fadeaway Bank Jump Shot',
 'Hook Shot']

In [16]:
list(set(shot_type_df['CFPARAMS']))

['Driving Finger Roll Layup Shot',
 'Putback Layup Shot',
 'Turnaround Fadeaway shot',
 'Alley Oop Layup shot',
 'Alley Oop Dunk Shot',
 'Driving Layup Shot',
 'Dunk Shot',
 'Hook Bank Shot',
 'Jump Shot',
 'Running Alley Oop Layup Shot',
 'Jump Bank Shot',
 'Driving Floating Bank Jump Shot',
 'Driving Bank Hook Shot',
 'Reverse Dunk Shot',
 'Driving Floating Jump Shot',
 'Running Alley Oop Dunk Shot',
 'Running Reverse Layup Shot',
 'Running Jump Shot',
 'Layup Shot',
 'Running Finger Roll Layup Shot',
 'Fadeaway Jump Shot',
 'Pullup Jump shot',
 'Tip Layup Shot',
 'Reverse Layup Shot',
 'Step Back Jump shot',
 'Driving Dunk Shot',
 'Tip Dunk Shot',
 'Finger Roll Layup Shot',
 'Cutting Finger Roll Layup Shot',
 'Running Pull-Up Jump Shot',
 'Driving Hook Shot',
 'Turnaround Bank Hook Shot',
 'Putback Dunk Shot',
 'Cutting Dunk Shot',
 'Floating Jump shot',
 'Driving Reverse Layup Shot',
 'Turnaround Jump Shot',
 'Cutting Layup Shot',
 'Running Layup Shot',
 'Step Back Bank Jump Shot',

In [17]:
shot_area_df = pd.DataFrame(shooters.data['ShotAreaPlayerDashboard'])
shot_area_df.head(10)

,BLKA,BLKA_RANK,CFID,CFPARAMS,EFG_PCT,EFG_PCT_RANK,FG3A,FG3A_RANK,FG3M,FG3M_RANK,...,PCT_AST_3PM,PCT_AST_3PM_RANK,PCT_AST_FGM,PCT_AST_FGM_RANK,PCT_UAST_2PM,PCT_UAST_2PM_RANK,PCT_UAST_3PM,PCT_UAST_3PM_RANK,PCT_UAST_FGM,PCT_UAST_FGM_RANK
0,12,7,49,Restricted Area,0.608,3,0,6,0,4,...,0.00,4,0.602,4,0.398,3,0.00,2,0.398,3
1,8,6,49,In The Paint (Non-RA),0.148,5,0,6,0,4,...,0.00,4,0.500,5,0.500,2,0.00,2,0.500,2
2,0,1,49,Mid-Range,0.111,6,1,5,0,4,...,0.00,4,0.000,6,1.000,1,0.00,2,1.000,1
3,0,1,49,Left Corner 3,0.640,2,75,3,32,3,...,1.00,1,1.000,1,0.000,4,0.00,2,0.000,5
4,0,1,49,Right Corner 3,0.650,1,113,2,49,2,...,1.00,1,1.000,1,0.000,4,0.00,2,0.000,5
5,1,5,49,Above the Break 3,0.491,4,271,1,89,1,...,0.91,3,0.910,3,0.000,4,0.09,1,0.090,4
6,0,1,49,Backcourt,0.000,7,2,4,0,4,...,0.00,4,0.000,6,0.000,4,0.00,2,0.000,5


After observing some of the potential features available, this shot probability model will go forward with the following features:
   - Shot distance
   - Angle to basket (0 degrees is straight on, 45 degrees is baseline)
   - Side of basket
   - Action type
   - Shooter 2PT FG%
   - Shooter 3PT FG%
   
Some of these features are available directly from the `shotchartdetail` endpoint, so the following cell contains a function to generate those features.

In [34]:
def feature_engineering(shotchart_df):
    """ feature_engineering calculates engineered
    features from the shotchart data

    @param shotchart_df (DataFrame): DataFrame containing
    shotchart data

    Returns:

        shotchart_df (DataFrame): DataFrame containing
        the engineered features
    """

    shotchart_df['ANGLE'] = abs(np.rad2deg(np.arctan2(shotchart_df['LOC_X'],
                                                      shotchart_df['LOC_Y'])))

    shotchart_df['SIDE'] = [1 if x >= 0 else 0 for x in shotchart_df['LOC_X']]

    shotchart_df = shotchart_df[['SHOT_DISTANCE', 'ANGLE', 'SIDE', 'ACTION_TYPE', 'SHOT_MADE_FLAG']]
    shotchart_df = pd.get_dummies(shotchart_df, prefix='AT', columns=['ACTION_TYPE'])
    return shotchart_df

In [35]:
shot_df = feature_engineering(shot_df)

In [36]:
shot_df

,SHOT_DISTANCE,ANGLE,SIDE,SHOT_MADE_FLAG,AT_Alley Oop Dunk Shot,AT_Cutting Dunk Shot,AT_Cutting Layup Shot,AT_Driving Dunk Shot,AT_Driving Finger Roll Layup Shot,AT_Driving Floating Jump Shot,...,AT_Putback Dunk Shot,AT_Putback Layup Shot,AT_Reverse Layup Shot,AT_Running Layup Shot,AT_Running Reverse Layup Shot,AT_Step Back Jump shot,AT_Tip Layup Shot,AT_Turnaround Fadeaway Bank Jump Shot,AT_Turnaround Fadeaway shot,AT_Turnaround Jump Shot
0,27,56.251467,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,16,49.513988,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,24,40.902716,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,29,29.647232,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,78.310631,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,24,59.511060,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,24,75.629998,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,10,32.932231,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,6,1.847610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,19.536655,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


The above handles the feature engineering of all desired features with the exception of the individual field goal percentages. It may be worth using these features to develop a baseline model. Gathering more data would be the next step. The next cell contains functions that pull shot chart data for an entire NBA regular season.

In [43]:
def pad_id(num):
    """ pad_id adds the requisite number of leading
    zeroes to a game number to form a valid game_id

    @param num (int): Regular season game number

    Returns:

        num_str (str): Regular season game number
        with leading zeroes
    """
    
    num_str = str(num)
    while len(num_str) < 4:
        num_str = '0' + num_str
        
    return num_str


def get_year_shotcharts(year):
    """ get_year_shotcharts pulls data from the
    shotchartdetail endpoint of the stats.nba.com API

    @param year (int): year corresponding to the year
    in which the season began. For example, the 2017-2018
    NBA season is represented as the year 2017.

    Returns:

        Saves a .csv file of the shotchart data to the
        current working directory with a name formatted
        as shotchart_[year].csv
    """

    year_sub = str(year)[-2:]

    base = '002' + year_sub + '0'

    all_df = pd.DataFrame({})

    for x in range(1, 1231):
        print('Game #' + str(x))
        game_id_here = base + pad_id(x)

        t0 = time.time()
        shots = player.Player(headers=HEADERS,
                      endpoint='shotchartdetail',
                      league_id=league_id,
                      player_id=player_id,
                      game_id=game_id_here,
                      season=season)

        shot_df = pd.DataFrame(shots.data['Shot_Chart_Detail'])
        delay = time.time() - t0
        print('Waiting ' + str(10*delay) + 's')
        time.sleep(delay)

        all_df = pd.concat([all_df, shot_df], axis=0).reset_index(drop=True)

    all_df.to_csv('shotchart_'+str(year)+'.csv', index=False)

In [44]:
#get_year_shotcharts(2017)

Game #1
Waiting 3.520689010620117s
Game #2
Waiting 6.249241828918457s
Game #3
Waiting 7.509241104125977s
Game #4
Waiting 6.663801670074463s
Game #5
Waiting 7.598671913146973s
Game #6
Waiting 28.055670261383057s
Game #7
Waiting 6.675169467926025s
Game #8
Waiting 6.96026086807251s
Game #9
Waiting 7.751979827880859s
Game #10
Waiting 7.45725154876709s
Game #11
Waiting 16.916940212249756s
Game #12
Waiting 6.5244221687316895s
Game #13
Waiting 7.082271575927734s
Game #14
Waiting 7.655818462371826s
Game #15
Waiting 6.226980686187744s
Game #16
Waiting 8.210930824279785s
Game #17
Waiting 6.927518844604492s
Game #18
Waiting 7.259171009063721s
Game #19
Waiting 6.956570148468018s
Game #20
Waiting 6.529967784881592s
Game #21
Waiting 8.207108974456787s
Game #22
Waiting 8.1181001663208s
Game #23
Waiting 6.927430629730225s
Game #24
Waiting 8.223507404327393s
Game #25
Waiting 7.968721389770508s
Game #26
Waiting 9.225499629974365s
Game #27
Waiting 10.483400821685791s
Game #28
Waiting 7.272851467132568s
G

Waiting 6.363890171051025s
Game #226
Waiting 6.968491077423096s
Game #227
Waiting 6.765880584716797s
Game #228
Waiting 7.275567054748535s
Game #229
Waiting 6.925899982452393s
Game #230
Waiting 6.1161088943481445s
Game #231
Waiting 6.826057434082031s
Game #232
Waiting 6.585662364959717s
Game #233
Waiting 7.190649509429932s
Game #234
Waiting 6.5771484375s
Game #235
Waiting 9.520289897918701s
Game #236
Waiting 12.454209327697754s
Game #237
Waiting 9.398472309112549s
Game #238
Waiting 17.51744031906128s
Game #239
Waiting 7.209630012512207s
Game #240
Waiting 18.070449829101562s
Game #241
Waiting 5.85618257522583s
Game #242
Waiting 7.197511196136475s
Game #243
Waiting 6.959249973297119s
Game #244
Waiting 6.246612071990967s
Game #245
Waiting 6.401810646057129s
Game #246
Waiting 9.540588855743408s
Game #247
Waiting 6.19675874710083s
Game #248
Waiting 8.221361637115479s
Game #249
Waiting 6.079239845275879s
Game #250
Waiting 6.872220039367676s
Game #251
Waiting 5.748329162597656s
Game #252
Waiti

Game #447
Waiting 7.2577714920043945s
Game #448
Waiting 6.871669292449951s
Game #449
Waiting 7.234489917755127s
Game #450
Waiting 23.17988872528076s
Game #451
Waiting 7.055099010467529s
Game #452
Waiting 8.159890174865723s
Game #453
Waiting 6.882779598236084s
Game #454
Waiting 7.107148170471191s
Game #455
Waiting 7.922329902648926s
Game #456
Waiting 6.645541191101074s
Game #457
Waiting 6.687359809875488s
Game #458
Waiting 6.963610649108887s
Game #459
Waiting 7.101991176605225s
Game #460
Waiting 6.912860870361328s
Game #461
Waiting 7.076129913330078s
Game #462
Waiting 6.8592000007629395s
Game #463
Waiting 6.712350845336914s
Game #464
Waiting 6.930501461029053s
Game #465
Waiting 7.046520709991455s
Game #466
Waiting 7.702069282531738s
Game #467
Waiting 9.839072227478027s
Game #468
Waiting 10.930187702178955s
Game #469
Waiting 7.842891216278076s
Game #470
Waiting 17.069730758666992s
Game #471
Waiting 7.474303245544434s
Game #472
Waiting 9.038200378417969s
Game #473
Waiting 6.85854911804199

Waiting 6.724710464477539s
Game #669
Waiting 6.251888275146484s
Game #670
Waiting 7.679188251495361s
Game #671
Waiting 7.230191230773926s
Game #672
Waiting 6.684727668762207s
Game #673
Waiting 7.341220378875732s
Game #674
Waiting 17.63171911239624s
Game #675
Waiting 7.005419731140137s
Game #676
Waiting 6.634809970855713s
Game #677
Waiting 6.631557941436768s
Game #678
Waiting 6.881048679351807s
Game #679
Waiting 7.0272111892700195s
Game #680
Waiting 6.908819675445557s
Game #681
Waiting 6.826622486114502s
Game #682
Waiting 7.134180068969727s
Game #683
Waiting 7.793769836425781s
Game #684
Waiting 7.149498462677002s
Game #685
Waiting 6.885190010070801s
Game #686
Waiting 6.920068264007568s
Game #687
Waiting 8.012771606445312s
Game #688
Waiting 7.012829780578613s
Game #689
Waiting 6.768310070037842s
Game #690
Waiting 7.135679721832275s
Game #691
Waiting 6.697888374328613s
Game #692
Waiting 18.436386585235596s
Game #693
Waiting 6.643059253692627s
Game #694
Waiting 7.216739654541016s
Game #695

Game #890
Waiting 8.679699897766113s
Game #891
Waiting 6.121432781219482s
Game #892
Waiting 6.479659080505371s
Game #893
Waiting 8.70913028717041s
Game #894
Waiting 7.126312255859375s
Game #895
Waiting 7.553749084472656s
Game #896
Waiting 8.34503173828125s
Game #897
Waiting 7.598309516906738s
Game #898
Waiting 7.365810871124268s
Game #899
Waiting 7.47136116027832s
Game #900
Waiting 7.271177768707275s
Game #901
Waiting 7.594971656799316s
Game #902
Waiting 7.280681133270264s
Game #903
Waiting 7.477619647979736s
Game #904
Waiting 7.262599468231201s
Game #905
Waiting 6.4791107177734375s
Game #906
Waiting 9.405090808868408s
Game #907
Waiting 7.446401119232178s
Game #908
Waiting 7.144362926483154s
Game #909
Waiting 6.56886100769043s
Game #910
Waiting 7.25513219833374s
Game #911
Waiting 6.626029014587402s
Game #912
Waiting 7.13104248046875s
Game #913
Waiting 7.550950050354004s
Game #914
Waiting 8.205690383911133s
Game #915
Waiting 6.647891998291016s
Game #916
Waiting 9.175739288330078s
Game #

Game #1109
Waiting 7.047157287597656s
Game #1110
Waiting 6.657240390777588s
Game #1111
Waiting 6.999599933624268s
Game #1112
Waiting 7.671072483062744s
Game #1113
Waiting 6.9591593742370605s
Game #1114
Waiting 6.808960437774658s
Game #1115
Waiting 7.074611186981201s
Game #1116
Waiting 6.799728870391846s
Game #1117
Waiting 7.5296783447265625s
Game #1118
Waiting 7.614128589630127s
Game #1119
Waiting 7.193689346313477s
Game #1120
Waiting 7.453041076660156s
Game #1121
Waiting 6.346139907836914s
Game #1122
Waiting 7.388758659362793s
Game #1123
Waiting 6.485269069671631s
Game #1124
Waiting 7.23038911819458s
Game #1125
Waiting 6.4868879318237305s
Game #1126
Waiting 7.103328704833984s
Game #1127
Waiting 7.7341508865356445s
Game #1128
Waiting 6.9870686531066895s
Game #1129
Waiting 6.6173577308654785s
Game #1130
Waiting 7.224447727203369s
Game #1131
Waiting 7.367191314697266s
Game #1132
Waiting 6.318578720092773s
Game #1133
Waiting 6.196620464324951s
Game #1134
Waiting 8.591639995574951s
Game #1

In [45]:
all_shots_df = pd.read_csv('shotchart_2017.csv')

In [46]:
all_shots_df

,ACTION_TYPE,EVENT_TYPE,GAME_DATE,GAME_EVENT_ID,GAME_ID,GRID_TYPE,HTM,LOC_X,LOC_Y,MINUTES_REMAINING,...,SHOT_ATTEMPTED_FLAG,SHOT_DISTANCE,SHOT_MADE_FLAG,SHOT_TYPE,SHOT_ZONE_AREA,SHOT_ZONE_BASIC,SHOT_ZONE_RANGE,TEAM_ID,TEAM_NAME,VTM
0,Driving Floating Jump Shot,Made Shot,20171017,7,21700001,Shot Chart Detail,CLE,-1,100,11,...,1,10,1,2PT Field Goal,Center(C),In The Paint (Non-RA),8-16 ft.,1610612738,Boston Celtics,BOS
1,Layup Shot,Missed Shot,20171017,9,21700001,Shot Chart Detail,CLE,-18,4,11,...,1,1,0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612739,Cleveland Cavaliers,BOS
2,Jump Shot,Missed Shot,20171017,12,21700001,Shot Chart Detail,CLE,-214,138,11,...,1,25,0,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612738,Boston Celtics,BOS
3,Jump Shot,Missed Shot,20171017,14,21700001,Shot Chart Detail,CLE,-131,83,11,...,1,15,0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612739,Cleveland Cavaliers,BOS
4,Cutting Layup Shot,Missed Shot,20171017,16,21700001,Shot Chart Detail,CLE,10,23,10,...,1,2,0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612738,Boston Celtics,BOS
5,Jump Shot,Made Shot,20171017,20,21700001,Shot Chart Detail,CLE,47,190,10,...,1,19,1,2PT Field Goal,Center(C),Mid-Range,16-24 ft.,1610612738,Boston Celtics,BOS
6,Jump Shot,Made Shot,20171017,24,21700001,Shot Chart Detail,CLE,217,56,10,...,1,22,1,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612739,Cleveland Cavaliers,BOS
7,Turnaround Fadeaway shot,Made Shot,20171017,29,21700001,Shot Chart Detail,CLE,-117,35,9,...,1,12,1,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612739,Cleveland Cavaliers,BOS
8,Driving Dunk Shot,Missed Shot,20171017,31,21700001,Shot Chart Detail,CLE,-11,6,9,...,1,1,0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612738,Boston Celtics,BOS
9,Running Layup Shot,Made Shot,20171017,33,21700001,Shot Chart Detail,CLE,-5,11,9,...,1,1,1,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612739,Cleveland Cavaliers,BOS
